# Introduction to Spark

In this notebook, you will be introduced to the Apache Spark libary for big data processing. There's a now a python package called `pyspark` which will load Spark for you. Tthe variable `sc` is a Spark context that lets you interact with the Spark runtime. Check that it is correctly initialised:

In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
print(sc)
print("Ready to go!")

<SparkContext master=local[*] appName=pyspark-shell>
Ready to go!


### Learning activity: Create RDDs

To analyse large datasets using Spark you will load them into Resilient Distributed Datasets (RDDs). There are a number of ways in which you can create RDDs. Use the `parallelize()` function to create one from a Python collection of elements ["Hello", "World", "!"] 

In [2]:
l = ["Hello", "World", "!"]
sc.parallelize(l)

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:480

Use the `textFile()` function to create an RDD from the file `war-and-peace.txt` and store it in a variable called `lines`. 

In [4]:
lines = sc.textFile("war-and-peace.txt")

### Learning activity: Basic RDD manipulation

Print the number of lines in War and Peace using the functions `count()`.

In [7]:
print(lines.count())

54223


Print the first 15 lines using the functions `take()`.

In [8]:
print(lines.take(15))

['                                      1869', '                                 WAR AND PEACE', '                                 by Leo Tolstoy', 'BK1', '                                 BOOK ONE: 1805', 'BK1|CH1', '  CHAPTER I', '', '  "Well, Prince, so Genoa and Lucca are now just family estates of the', "Buonapartes. But I warn you, if you don't tell me that this means war,", 'if you still try to defend the infamies and horrors perpetrated by', 'that Antichrist- I really believe he is Antichrist- I will have', 'nothing more to do with you and you are no longer my friend, no longer', "my 'faithful slave,' as you call yourself! But how do you do? I see", 'I have frightened you- sit down and tell me all the news."']


### Learning activity: `filter()` and `map()` and `distinct()`

Lets apply some transformations onto RDDs. The following helper function will be useful to select the words from a line.

In [9]:
# A helper function to compute the list of words in a line of text
import re
def get_words(line):
    return re.compile('\w+').findall(line)

print(get_words("This, is a test!"))

['This', 'is', 'a', 'test']


Use `filter()` to count the number of lines which mention `war` and the number of lines which mention `peace`.

In [42]:
# How often are war and peace mentioned?
war = lines.filter(lambda a: "war" in get_words(a))
print(war.count())
peace = lines.filter(lambda a: "peace" in get_words(a))
print(peace.count())

265
104


Use `map()` to capitalise each line in the RDD, and print the first 15 capitalized lines.

In [27]:
# Capitalize each line in the RDD
print(lines.map(lambda a: a.upper()).take(15))

['                                      1869', '                                 WAR AND PEACE', '                                 BY LEO TOLSTOY', 'BK1', '                                 BOOK ONE: 1805', 'BK1|CH1', '  CHAPTER I', '', '  "WELL, PRINCE, SO GENOA AND LUCCA ARE NOW JUST FAMILY ESTATES OF THE', "BUONAPARTES. BUT I WARN YOU, IF YOU DON'T TELL ME THAT THIS MEANS WAR,", 'IF YOU STILL TRY TO DEFEND THE INFAMIES AND HORRORS PERPETRATED BY', 'THAT ANTICHRIST- I REALLY BELIEVE HE IS ANTICHRIST- I WILL HAVE', 'NOTHING MORE TO DO WITH YOU AND YOU ARE NO LONGER MY FRIEND, NO LONGER', "MY 'FAITHFUL SLAVE,' AS YOU CALL YOURSELF! BUT HOW DO YOU DO? I SEE", 'I HAVE FRIGHTENED YOU- SIT DOWN AND TELL ME ALL THE NEWS."']


Use `flatMap()` to create an RDD of the words in War and Peace and count the number of words.

In [28]:
# Split each line into words using get_words()
words = lines.flatMap(lambda a: get_words(a))
print(words.count())

573322


Finally, use `distinct()` to count the number of different words in the RDD.

In [47]:
# Count the number of distinct words
distinct_words = words.distinct()
print(distinct_words.count())

19206


### Learning activity: Set like transformations

Use the functions `union()` and `intersection()` to create RDDs of lines with either war or peace mentioned, and both war and peace being mentioned. Count how many lines of each type there are and print some examples.

In [43]:
union = war.union(peace)
print(union.count())
intersect = war.intersection(peace)
print(intersect.count())

369
7


### Learning activity: `reduce()`

You have already seen three actions: `collect()` which returns all elements in the RDD, `take(n)`, which return the first `n` elements of the RDD, and `count()` which returns the number of elements in the RDD.

The action `reduce()` takes as input a function which collapses two elements into one. Use it to find the longest word in War and Peace.

In [37]:
longest_word = words.reduce(lambda a,b: max(a, b, key=len))
print(longest_word)

characteristically


### Bonus activity: merging filters

Find all the lines that mention both war and peace **without** using `intersection()`.

In [44]:
war_and_peace = lines.filter(lambda a: "war" in get_words(a) and "peace" in get_words(a))
print(war_and_peace.count())

7


### Bonus learning activity: Finding proper nouns

The Python function `str.istitle()` returns `True` if the string `str` is titlecased: the first character is uppercase and others are lowercase. Use it to:
* Find the set of distinct words in War and Peace which are titlecased
* Find the set of distinct words in War and Peace which are not titlecased

In [53]:
titlecased = words.filter(lambda a: a.istitle()).distinct()
print(titlecased.count())
not_titlecased = words.filter(lambda a: not a.istitle()).distinct()
print(not_titlecased.count())

3068
16138


The Python function `str.lower` returns a string with all characters of `str` lowercase. Use it, along with your previously generated RDD to find the set of words in War and Peace which only appear titlecased.

In [64]:
proper_nouns = titlecased.subtract(titlecased.map(lambda a: a.lower()).intersection(not_titlecased).map(lambda a: a.title()))
print(proper_nouns.count())
#print(proper_nouns.take(1000))

1548


# Key/Value pairs in Spark

### Learning activity: WordCount in Spark

Use the functions `flatMap()` and `reduceByKey()` to count the number of occurences of each word in War and Peace, and print the count of five words.

In [76]:
word_map = words.map(lambda a: (a.lower(), 1))
word_count = word_map.reduceByKey(lambda a,b: a + b)
print(word_count.take(5))

[('peace', 110), ('leo', 1), ('tolstoy', 10), ('bk1', 29), ('book', 71)]


### Learning activity: using `groupByKey()`

Reimplement the above word count using `groupByKey()` instead of `reduceByKey()`

In [81]:
word_count_2 = word_map.groupByKey().mapValues(lambda a: len(a))
print(word_count_2.take(5))

[('peace', 110), ('leo', 1), ('tolstoy', 10), ('bk1', 29), ('book', 71)]


### Bonus learning activity: computing the average of each key

The pair RDD defined below `word_line_pairs` has an element for each line in War and Peace with as key the first word, and as value the line itself. Use it to compute the average length of each line for each starting word.

In [ ]:
word_line_pairs = lines.filter(lambda line: len(get_words(line)) > 0).map(lambda line: (get_words(line)[0], line))     